# Text Summarization

You task is to prepare a model that is suitable for text summarization.
We gonna use CNN_dailymail dataset

Источники
https://discuss.huggingface.co/t/how-to-turn-wandb-off-in-trainer/6237    
https://discuss.huggingface.co/t/torch-cuda-outofmemoryerror-cuda-out-of-memory-tried-to-allocate-256-00-mib-gpu-0-39-56-gib-total-capacity-37-84-gib-already-allocated-242-56-mib-free-37-96-gib-reserved-in-total-by-pytorch/42381    
https://huggingface.co/docs/transformers/tasks/summarization    
https://cameronrwolfe.substack.com/p/t5-text-to-text-transformers-part
https://github.com/sabre-code/text-summarisation/blob/main/text_summarisation.ipynb    
https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/trainer#transformers.Trainer


In [1]:
!pip install datasets transformers evaluate rouge_score
from datasets import load_dataset
from evaluate import load

import math
import torch
from torch import nn

dataset = load_dataset("abisee/cnn_dailymail", '3.0.0')
metric = load("rouge")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=168e6063d708d379c32b8caedec3795a852ba1e02451047dd9788dc17df4e319
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


2024-06-17 14:09:25.095360: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 14:09:25.095513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 14:09:25.256986: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

### Now let's see an example of tokenizing data with transformers tokinizer by HuggingFace 🤗

#### Please note that I'm using different dataset with different layout for the example


In [2]:
from transformers import AutoTokenizer

raw_datasets = load_dataset("xsum")
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_input_ids"] = model_inputs["labels"]
    return model_inputs
tokenizer = AutoTokenizer.from_pretrained('t5-small')


/opt/conda/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

### This next line of code gonan take some time.
To familarize yourself with this example, you might just process a few examples

In [3]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, num_proc=4)
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels", "decoder_input_ids"])

Map (num_proc=4):   0%|          | 0/204045 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/11332 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/11334 [00:00<?, ? examples/s]

## Now you have to prepare your dataset
Use any tokenization methods you'd like <br>
If you are using guides please cite your sourse

## Declare  your model (6 pts)
Use encoder-decoder transformer architecure of any size <br>
You can lookup for a T5 architecure





In [4]:
import torch.nn as nn
from transformers import PreTrainedModel, PretrainedConfig

### CONFIG ###
class MyModelConfig(PretrainedConfig):
    def __init__(self, vocab_size=32128, d_model=512, num_heads=8, num_layers=6, d_ff=2048, **kwargs):
        super().__init__(**kwargs)
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.d_ff = d_ff

### MultiHeadAttention ###
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = config.num_heads
        self.head_dim = config.d_model // config.num_heads
        self.scale = self.head_dim ** -0.5

        self.qkv_proj = nn.Linear(config.d_model, config.d_model * 3)
        self.out_proj = nn.Linear(config.d_model, config.d_model)

    def forward(self, hidden_states, mask=None):
        batch_size, seq_length, d_model = hidden_states.size()
        qkv = self.qkv_proj(hidden_states)
        qkv = qkv.view(batch_size, seq_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3).contiguous()
        q, k, v = qkv.chunk(3, dim=-1)

        scores = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        if mask is not None:
            mask = mask[:, None, None, :].to(scores.dtype)
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        context = torch.matmul(attn, v)
        context = context.permute(0, 2, 1, 3).contiguous().view(batch_size, seq_length, d_model)

        return self.out_proj(context)

### CrossAttention ###
class CrossAttention(nn.Module):
    def __init__(self, config):
        super(CrossAttention, self).__init__()
        self.num_heads = config.num_heads
        self.head_dim = config.d_model // config.num_heads
        self.scale = self.head_dim ** -0.5

        self.q_proj = nn.Linear(config.d_model, config.d_model)
        self.kv_proj = nn.Linear(config.d_model, config.d_model * 2)
        self.out_proj = nn.Linear(config.d_model, config.d_model)

    def forward(self, hidden_states, encoder_hidden_states, mask=None):
        batch_size, seq_length, d_model = hidden_states.size()
        q = self.q_proj(hidden_states)
        kv = self.kv_proj(encoder_hidden_states)
        kv = kv.view(batch_size, -1, self.num_heads, 2 * self.head_dim)
        kv = kv.permute(0, 2, 1, 3).contiguous()
        k, v = kv.chunk(2, dim=-1)

        q = q.view(batch_size, seq_length, self.num_heads, self.head_dim)
        q = q.permute(0, 2, 1, 3).contiguous()

        scores = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        if mask is not None:
            mask = mask[:, None, None, :].to(scores.dtype)
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        context = torch.matmul(attn, v)
        context = context.permute(0, 2, 1, 3).contiguous().view(batch_size, seq_length, d_model)

        return self.out_proj(context)

### MaskedAttention ###
class MaskedAttention(nn.Module):
    def __init__(self, config):
        super(MaskedAttention, self).__init__()
        self.num_heads = config.num_heads
        self.head_dim = config.d_model // config.num_heads
        self.scale = self.head_dim ** -0.5

        self.qkv_proj = nn.Linear(config.d_model, config.d_model * 3)
        self.out_proj = nn.Linear(config.d_model, config.d_model)

    def forward(self, hidden_states, mask=None):
        batch_size, seq_length, d_model = hidden_states.size()
        qkv = self.qkv_proj(hidden_states)
        qkv = qkv.view(batch_size, seq_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3).contiguous()
        q, k, v = qkv.chunk(3, dim=-1)

        scores = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        if mask is not None:
            mask = mask[:, None, None, :].to(scores.dtype)
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = torch.softmax(scores, dim=-1)
        context = torch.matmul(attn, v)
        context = context.permute(0, 2, 1, 3).contiguous().view(batch_size, seq_length, d_model)

        return self.out_proj(context)

### EncoderBlock ###
class EncoderBlock(nn.Module):
    def __init__(self, config):
        super(EncoderBlock, self).__init__()
        self.self_attn = MultiHeadAttention(config)
        self.ffn = nn.Sequential(
            nn.Linear(config.d_model, config.d_ff),
            nn.ReLU(),
            nn.Linear(config.d_ff, config.d_model)
        )
        self.ln1 = nn.LayerNorm(config.d_model)
        self.ln2 = nn.LayerNorm(config.d_model)

    def forward(self, hidden_states, mask=None):
        attn_output = self.self_attn(hidden_states, mask)
        hidden_states = self.ln1(hidden_states + attn_output)
        ffn_output = self.ffn(hidden_states)
        hidden_states = self.ln2(hidden_states + ffn_output)
        return hidden_states

### Encoder ###
class Encoder(nn.Module):
    def __init__(self, config):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(config.vocab_size, config.d_model)
        self.blocks = nn.ModuleList([EncoderBlock(config) for _ in range(config.num_layers)])
        self.ln = nn.LayerNorm(config.d_model)

    def forward(self, input_ids, mask=None):
        hidden_states = self.embedding(input_ids)
        for block in self.blocks:
            hidden_states = block(hidden_states, mask)
        return self.ln(hidden_states)

### DecoderBlock ###
class DecoderBlock(nn.Module):
    def __init__(self, config):
        super(DecoderBlock, self).__init__()
        self.self_attn = MaskedAttention(config)
        self.cross_attn = CrossAttention(config)
        self.ffn = nn.Sequential(
            nn.Linear(config.d_model, config.d_ff),
            nn.ReLU(),
            nn.Linear(config.d_ff, config.d_model)
        )
        self.ln1 = nn.LayerNorm(config.d_model)
        self.ln2 = nn.LayerNorm(config.d_model)
        self.ln3 = nn.LayerNorm(config.d_model)

    def forward(self, hidden_states, encoder_output, self_mask=None, cross_mask=None):
        attn_output = self.self_attn(hidden_states, self_mask)
        hidden_states = self.ln1(hidden_states + attn_output)
        cross_attn_output = self.cross_attn(hidden_states, encoder_output, cross_mask)
        hidden_states = self.ln2(hidden_states + cross_attn_output)
        ffn_output = self.ffn(hidden_states)
        hidden_states = self.ln3(hidden_states + ffn_output)
        return hidden_states

### Decoder ###
class Decoder(nn.Module):
    def __init__(self, config):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(config.vocab_size, config.d_model)
        self.blocks = nn.ModuleList([DecoderBlock(config) for _ in range(config.num_layers)])
        self.ln = nn.LayerNorm(config.d_model)
        self.output_proj = nn.Linear(config.d_model, config.vocab_size)

    def forward(self, input_ids, encoder_output, self_mask=None, cross_mask=None):
        if input_ids is None:
            input_ids = torch.zeros((encoder_output.size(0), encoder_output.size(1)), dtype=torch.long).to(encoder_output.device)
        hidden_states = self.embedding(input_ids)
        for block in self.blocks:
            hidden_states = block(hidden_states, encoder_output, self_mask, cross_mask)
        return self.output_proj(self.ln(hidden_states))


### MyModel  ###
class MyModel(PreTrainedModel):
    config_class = MyModelConfig

    def __init__(self, config):
        super(MyModel, self).__init__(config)
        self.encoder = Encoder(config)
        self.decoder = Decoder(config)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None):
        encoder_output = self.encoder(input_ids, attention_mask)
        if decoder_input_ids is None:
            decoder_input_ids = torch.zeros_like(input_ids)
        decoder_output = self.decoder(decoder_input_ids, encoder_output, decoder_attention_mask, attention_mask)
        
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            loss = loss_fct(decoder_output.view(-1, self.config.vocab_size), labels.view(-1))
            return {"loss": loss, "logits": decoder_output}
        else:
            return {"logits": decoder_output}


### Easy way
If you are struggling with some modules, you can use PyTorch layers:
```
nn.TransformerEncoderLayer   - 0.2 pts penalty
nn.TransformerDecoderLayer   - 0.2 pts penalty
nn.TransformerEncoder        - 0.5 pts penalty
nn.TransformerDecoder        - 0.5 pts penalty
nn.Transformer               - 1.0 pts penalty   

```



## Train your model (2 pts)

You are free to use any library, including the HuggingFace🤗

In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Adjust batch size as needed
    per_device_eval_batch_size=4,   # Adjust batch size as needed
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    save_steps=10_000,
    eval_steps=1_000,
    logging_dir="./logs",
    logging_steps=500,
    no_cuda=True,  # Ensure CUDA is not used
)

# Initialize your custom model with the config
config = MyModelConfig()
model = MyModel(config)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

В идеальном мире оно дообучилось     
¯\_(ツ)_/¯


In [ ]:
results = trainer.evaluate()
print(results)


## Fine tune a model from HuggingFace (2 pts)

Use a [guide](https://huggingface.co/docs/transformers/tasks/summarization) from HuggingFace🤗

In [11]:
from transformers import AutoModelForSeq2SeqLM
### FINETUNE ###
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
)

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

trainer.train()
results = trainer.evaluate()
print(results)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

### Compare your models (2 pts)

Use rouge metric. See the example above. <br>


Feed your own texts to both model and see the result


Add a conclusion

In [ ]:
### METRICS EVALUATION ###

### My conclusion: